In [1]:
from modules.settings import global_mmf_settings as gset
from modules import parallel_Y_M_scaling_reln as ymsr
from modules.settings import global_mmf_settings as gset
from cosmology import cosmo_fn
import multiprocessing as mp
from scipy.optimize import curve_fit
import bces.bces as bces
import time

%pylab notebook

def return_log_err(gauss_mean,gauss_err,num_samples=20000,ignore_negatives=True):
    logerr=np.zeros_like(gauss_mean)
    for idx, mu in enumerate(gauss_mean):
        x=np.random.normal(mu,gauss_err[idx],num_samples)
        if ignore_negatives:
            while np.any(x<0):
                neg_idx=np.where(x<0)[0]
                x[neg_idx]=np.random.normal(mu,gauss_err[idx],np.size(neg_idx))
        logerr[idx]=np.std(log10(x))
    return logerr

Populating the interactive namespace from numpy and matplotlib


# Initialize MMF analysis

In [2]:
outpath="../results/planck_esz/"
gset.setup_mmf_config(outpath=outpath,chmin=100.,xsize=10.,result_midfix="random_thet500")
figpath=gset.mmfset.paths["result_figs"]
pmmf=ymsr.Y_M_scaling()
xsz_cat=pmmf.xsz_cat

def wrap_eval_Y500_xray_prior(idx):
    result=pmmf.eval_Y500_xray_prior(idx)
    return result

def parallel_eval_Y500_xray_prior(numprocs):
    pool=mp.Pool(processes=numprocs)
    result=pool.map(wrap_eval_Y500_xray_prior,pmmf.idx_list)
    pool.close()
    pool.join()
    return np.array(result)

def wrap_eval_Y500_blind(idx):
    result=pmmf.eval_Y500_blind(idx)
    return result

def parallel_eval_Y500_blind(numprocs):
    pool=mp.Pool(processes=numprocs)
    result=pool.map(wrap_eval_Y500_blind,pmmf.idx_list)
    pool.close()
    pool.join()
    return np.array(result)

# MMF analysis with X-ray prior on location, size and temperature of cluster

In [ ]:
start=time.time()
result=parallel_eval_Y500_xray_prior(8)
print time.time()-start

dtype=["idx","theta500","T500","YSZ_500","YSZ_500_err","YSZ_500_Tc","YSZ_500_err_Tc"]
fxsz_cat={}
for i, d in enumerate(dtype):
    fxsz_cat[d]=result[:,i]

66.6810600758


# Blind MMF analysis

In [ ]:
start=time.time()
bresult=parallel_eval_Y500_blind(8)
print time.time()-start
dtype=["idx","theta500","T500","YSZ_500","YSZ_500_err","YSZ_500_Tc","YSZ_500_Tc_err"]
bxsz_cat={}
for i, d in enumerate(dtype):
    bxsz_cat[d]=bresult[:,i]

# Results xray priors

## Figures for null tests

In [ ]:
x=logspace(-5,-3.3,100)
figure()
lbl="X-ray priors"
errorbar(xsz_cat["YSZ_500"],fxsz_cat["YSZ_500"],xerr=xsz_cat["YSZ_500_err"],yerr=fxsz_cat["YSZ_500_err"],fmt="o",label=lbl)
lbl="Blind"
errorbar(xsz_cat["YSZ_500"],bxsz_cat["YSZ_500"],xerr=xsz_cat["YSZ_500_err"],yerr=bxsz_cat["YSZ_500_err"],fmt="o",label=lbl)


plot(x,x,"k--")
grid(which="both")
xlabel("$Y^{catalogue}_{500}$",fontsize=10)
ylabel("$Y^{estimated}_{500}$",fontsize=10)
loglog()
ylim(5e-6,1e-3)
xlim(5e-6,1e-3)
legend(loc=0)
figname=figpath + "compare_Y500.pdf"
savefig(figname,bbox_inches="tight")

In [ ]:
figure()
plot(fxsz_cat["YSZ_500_err"]/xsz_cat["YSZ_500_err"])
#axhline(1,ls="--",c="k")
axhline(1/1.414,ls="--",c="b",label="$1/\sqrt{2}=$" + str(round(1/sqrt(2.),3)))
mu=mean(np.array(fxsz_cat["YSZ_500_err"]/xsz_cat["YSZ_500_err"]))
axhline(mu,label="mean = "+ str(round(mu,3)),c="r")
legend(loc=0)
ylabel("$\sigma_{Y^{est.}_{500}}/\sigma_{Y^{cat.}_{500}}$",fontsize=12)
grid()
figname=figpath + "compare_Y500_err.pdf"
savefig(figname,bbox_inches="tight")

In [ ]:
figure()
scatter(xsz_cat["T500"],fxsz_cat["YSZ_500_Tc"]/fxsz_cat["YSZ_500"],label="$Y^{T=T_c}_{500}/Y^{T=0}_{500}$")
scatter(xsz_cat["T500"],fxsz_cat["YSZ_500_err_Tc"]/fxsz_cat["YSZ_500_err"],label="$\sigma^{T=T_c}_{Y_{500}}/\sigma^{T=0}_{Y_{500}}$")
plot(xsz_cat["T500"],1+0.07*xsz_cat["T500"]/5.,"k--",label=r"$1 + 0.07 \frac{T_{500}}{5.}$")
xlabel('$T_{500}$ keV')
ylabel('$Y^{T=T_c}_{500}/Y^{T=0}_{500}$')
grid()
legend(loc=0)
figname=figpath + "Y500_T0_vs_Y500_Tc.pdf"
savefig(figname,bbox_inches="tight")



# Ratio of SNR
qTT=fxsz_cat["YSZ_500_Tc"]/fxsz_cat["YSZ_500_err_Tc"]
qT0=fxsz_cat["YSZ_500"]/fxsz_cat["YSZ_500_err"]

fn=lambda x,a: 1.+a*x/5.
slope,slope_err=curve_fit(fn,xsz_cat["T500"],qTT/qT0)

figure()
plot(xsz_cat["T500"],qTT/qT0,"o")
#plot(xsz_cat["T500"],1+0.07*xsz_cat["T500"]/5.,"r-",label=r"$1 + 0.07 \frac{T_{500}}{5.}$")
plot(xsz_cat["T500"],fn(xsz_cat["T500"],slope),"k--",label="Fit: " + "$1 + $" + str(round(slope,4)) + r"$\frac{T_{500}}{5.}$")
xlabel('$T_{500}$ keV')
ylabel('$r=q_{T=T_c}/q_{T=0}$')
grid()
legend(loc=0)
figname=figpath + "SNR_qT0_qTT.pdf"

## Deriving $Y^{SZ}_{500} - Y^X_{500}$ scalinng law

In [ ]:
xdata=log10(xsz_cat["YX_500"]/1e-4)
xerr=return_log_err(xsz_cat["YX_500"]/1e-4,xsz_cat["YX_500_err"]/1e-4)

# Catalogue
ydata=log10(xsz_cat["YSZ_500"])
yerr=return_log_err(xsz_cat["YSZ_500"],xsz_cat["YSZ_500_err"])
acat,bcat,erracat,errbcat,covab=bces.bcesp(xdata,xerr,ydata,yerr,np.zeros_like(xdata),100000)
print acat[3],erracat[3],bcat[3],errbcat[3]


# T=0
ydata=log10(fxsz_cat["YSZ_500"])
yerr=return_log_err(fxsz_cat["YSZ_500"],fxsz_cat["YSZ_500_err"])
aT0,bT0,erraT0,errbT0,covab=bces.bcesp(xdata,xerr,ydata,yerr,np.zeros_like(xdata),100000)
print aT0[3],erraT0[3],bT0[3],errbT0[3]

# T=T_c
ydata=log10(fxsz_cat["YSZ_500_Tc"])
yerr=return_log_err(fxsz_cat["YSZ_500_Tc"],fxsz_cat["YSZ_500_err_Tc"])
aTT,bTT,erraTT,errbTT,covab=bces.bcesp(xdata,xerr,ydata,yerr,np.zeros_like(xdata),100000)
print aTT[3],erraTT[3],bTT[3],errbTT[3]

In [ ]:
figure()
errorbar(xsz_cat["YX_500"],xsz_cat["YSZ_500"],xerr=xsz_cat["YX_500_err"],yerr=xsz_cat["YSZ_500_err"],fmt='o',ms=5,label="Catalogue")
errorbar(xsz_cat["YX_500"]+0.02*xsz_cat["YX_500"],fxsz_cat["YSZ_500"],xerr=xsz_cat["YX_500_err"],yerr=fxsz_cat["YSZ_500_err"],fmt='o',ms=5,label="Estimated $(T=0)$")
tempx=linspace(5e-6,5e-3,1000)
lbl="$A_{cat}$ = " + str(round(bcat[3],2)) + "$\pm$" + str(round(errbcat[3],3)) 
lbl=lbl + " ; $B_{cat}$ = " +str(round(acat[3],2))  + "$\pm$" + str(round(erracat[3],3))
plot(tempx,(10.**bcat[3])*(tempx/1e-4)**acat[3],label=lbl,zorder=1)

lbl="$A_{T=0}$ = " + str(round(bT0[3],2)) + "$\pm$" + str(round(errbT0[3],3)) 
lbl=lbl + " ; $B_{T=0}$ = " +str(round(aT0[3],2))  + "$\pm$" + str(round(erraT0[3],3))
plot(tempx,(10.**bT0[3])*(tempx/1e-4)**aT0[3],label=lbl,zorder=3)
ylim(5e-6,1e-3)
xlim(5e-6,1e-3)
loglog()
legend(loc=0)
ylabel("$Y^{SZ}_{500}$",fontsize=10)
xlabel("$Y^X_{500}$",fontsize=10)
grid(which="both",alpha=0.4)
figname=figpath + "YSZ500_YX500_cat_vs_est.pdf"
savefig(figname,bbox_inches="tight")

In [ ]:
figure()
errorbar(xsz_cat["YX_500"],fxsz_cat["YSZ_500"],xerr=xsz_cat["YX_500_err"],yerr=fxsz_cat["YSZ_500_err"],fmt='o',ms=5,label="Estimated $(T=0)$")
errorbar(xsz_cat["YX_500"],fxsz_cat["YSZ_500_Tc"],xerr=xsz_cat["YX_500_err"],yerr=fxsz_cat["YSZ_500_err_Tc"],fmt='o',ms=5,label="Estimated $(T=T_c)$")
tempx=linspace(5e-6,5e-3,1000)
lbl="$A_{T=0}$ = " + str(round(bT0[3],2)) + "$\pm$" + str(round(errbT0[3],3)) 
lbl=lbl + " ; $B_{T=0}$ = " +str(round(aT0[3],2))  + "$\pm$" + str(round(erraT0[3],3))
plot(tempx,(10.**bT0[3])*(tempx/1e-4)**aT0[3],label=lbl,zorder=3)

lbl="$A_{T=T_c}$ = " + str(round(bTT[3],2)) + "$\pm$" + str(round(errbTT[3],3)) 
lbl=lbl + " ; $B_{T=T_c}$ = " +str(round(aTT[3],2))  + "$\pm$" + str(round(erraTT[3],3))
plot(tempx,(10.**bTT[3])*(tempx/1e-4)**aTT[3],label=lbl,zorder=5)

ylim(5e-6,1e-3)
xlim(5e-6,1e-3)
loglog()
legend(loc=0)
ylabel("$Y^{SZ}_{500}$",fontsize=10)
xlabel("$Y^X_{500}$",fontsize=10)
grid(which="both",alpha=0.4)
figname=figpath + "YSZ500_YX500_T0_vs_TT.pdf"
savefig(figname,bbox_inches="tight")

## Deriving $Y^{SZ}_{500} - M^X_{500}$ scalinng law

In [ ]:
# # This produces numbers in row 2 of Table 2 from Planck early results XI (A&A 536 A11 2011)
# Ezgamma=cosmo_fn.Ez(xsz_cat["z"])**(-2./3.)
# This produces numbers in Table A.1 from Planck Collaboration: Cosmology from SZ cluster counts (2013)
Ezgamma=cosmo_fn.Ez(xsz_cat["z"])**(-2./3.)/1e-4

xdata=log10(xsz_cat["M500"]/6.)
xerr=return_log_err(xsz_cat["M500"]/6.,xsz_cat["M500_err"]/6.)

# Catalogue
ydata=log10(xsz_cat["YSZ_500"]*Ezgamma)
yerr=return_log_err(xsz_cat["YSZ_500"]*Ezgamma,xsz_cat["YSZ_500_err"]*Ezgamma)
acat,bcat,erracat,errbcat,covab=bces.bcesp(xdata,xerr,ydata,yerr,np.zeros_like(xdata),100000)
print acat[3],erracat[3],bcat[3],errbcat[3]


# T=0
ydata=log10(fxsz_cat["YSZ_500"]*Ezgamma)
yerr=return_log_err(fxsz_cat["YSZ_500"]*Ezgamma,fxsz_cat["YSZ_500_err"]*Ezgamma)
aT0,bT0,erraT0,errbT0,covab=bces.bcesp(xdata,xerr,ydata,yerr,np.zeros_like(xdata),100000)
print aT0[3],erraT0[3],bT0[3],errbT0[3]

# T=T_c
ydata=log10(fxsz_cat["YSZ_500_Tc"]*Ezgamma)
yerr=return_log_err(fxsz_cat["YSZ_500_Tc"]*Ezgamma,fxsz_cat["YSZ_500_err_Tc"]*Ezgamma)

aTT,bTT,erraTT,errbTT,covab=bces.bcesp(xdata,xerr,ydata,yerr,np.zeros_like(xdata),100000)
print aTT[3],erraTT[3],bTT[3],errbTT[3]

In [ ]:
M500=linspace(2,24,1000)
norm=1e-4
figure()
errorbar(xsz_cat["M500"]*1e14,Ezgamma*xsz_cat["YSZ_500"]*norm,yerr=xsz_cat["YSZ_500_err"]*Ezgamma*norm,fmt="o",label="Catalogue")
errorbar(xsz_cat["M500"]*1e14,Ezgamma*fxsz_cat["YSZ_500"]*norm,yerr=fxsz_cat["YSZ_500_err"]*Ezgamma*norm,fmt="o",label="Estimated $(T=0)$")

lbl=r"$A_{cat}$ = " + str(round(bcat[3],2)) + r"$\pm$" + str(round(errbcat[3],2)) 
lbl=lbl + r" ; $\alpha_{cat}$ = " +str(round(acat[3],2))  + r"$\pm$" + str(round(erracat[3],2))
plot(M500*1e14,(10.**bcat[3])*((M500/6.)**acat[3])*norm,label=lbl)

lbl=r"$A_{T=0}$ = " + str(round(bT0[3],2)) + "$\pm$" + str(round(errbT0[3],2)) 
lbl=lbl + r" ; $\alpha_{T=0}$ = " +str(round(aT0[3],2))  + "$\pm$" + str(round(erraT0[3],2))
plot(M500*1e14,(10.**bT0[3])*((M500/6.)**aT0[3])*norm,label=lbl)

ylabel(r"$E(z)^{-2/3} D^2_A Y_{500}$")
xlabel(r"$M_{500}$" + " " + "$M_{\odot}$")
lbl=r"$E(z)^{-2/3} D^2_A Y_{500} = 10^A \left[ \frac{M_{500}}{6 \times 10^{14} M_{\odot}}\right]^\alpha$"
legend(loc=0,title=lbl,fontsize=8)
loglog()
grid(which="both",alpha=0.6)
figname=figpath + "Y500_vs_M500_compare_with_cat.pdf"
savefig(figname,bbox_inches="tight")

In [ ]:
M500=linspace(2,24,1000)
T500=cosmo_fn.convert_M500_T500(M500,0.)
correction=1+0.07*(T500/5.)

norm=1e-4
figure()
errorbar(xsz_cat["M500"]*1e14,Ezgamma*fxsz_cat["YSZ_500"]*norm,yerr=fxsz_cat["YSZ_500_err"]*Ezgamma*norm,fmt="o",label="Estimated $(T=0)$")
errorbar(xsz_cat["M500"]*1e14,Ezgamma*fxsz_cat["YSZ_500_Tc"]*norm,yerr=fxsz_cat["YSZ_500_err_Tc"]*Ezgamma*norm,fmt="o",label="Estimated $(T=T_c)$")

lbl=r"$A_{T=0}$ = " + str(round(bT0[3],2)) + "$\pm$" + str(round(errbT0[3],2)) 
lbl=lbl + r" ; $\alpha_{T=0}$ = " +str(round(aT0[3],2))  + "$\pm$" + str(round(erraT0[3],2))
plot(M500*1e14,(10.**bT0[3])*((M500/6.)**aT0[3])*norm,label=lbl)

lbl=r"$A_{T=T}$ = " + str(round(bTT[3],2)) + r"$\pm$" + str(round(errbTT[3],2)) 
lbl=lbl + r" ; $\alpha_{T=T_c}$ = " +str(round(aTT[3],2))  + r"$\pm$" + str(round(erraTT[3],2))
plot(M500*1e14,(10.**bTT[3])*((M500/6.)**aTT[3])*norm,label=lbl)

plot(M500*1e14,(10.**bT0[3])*((M500/6.)**aT0[3])*norm*correction,"k--",label="Theory")

ylabel(r"$E(z)^{-2/3} D^2_A Y_{500}$")
xlabel(r"$M_{500}$" + " " + "$M_{\odot}$")
lbl=r"$E(z)^{-2/3} D^2_A Y_{500} = 10^A \left[ \frac{M_{500}}{6 \times 10^{14} M_{\odot}}\right]^\alpha$"
legend(loc=0,title=lbl,fontsize=8)
loglog()
grid(which="both",alpha=0.6)
figname=figpath + "Y500_vs_M500_rSZ_correction.pdf"
savefig(figname,bbox_inches="tight")